<a href="https://colab.research.google.com/github/Beluuuuuuga/python_machine_learning_2nd_edition/blob/master/kaggle_ion_switching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Google driveにファイル保存する用
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# 4/wwEVHBbLp_T5iqhskY-swvFHQ_O_x3PvyYcgC8mQkXYW4H14lP9mwa0

In [7]:
# kaggle API使用
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!ls ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
kaggle.json


In [8]:
# 使用を確認
!kaggle competitions list

ref                                                                deadline             category             reward  teamCount  userHasEntered  
-----------------------------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
digit-recognizer                                                   2030-01-01 00:00:00  Getting Started   Knowledge       2290            True  
titanic                                                            2030-01-01 00:00:00  Getting Started   Knowledge      16542            True  
house-prices-advanced-regression-techniques                        2030-01-01 00:00:00  Getting Started   Knowledge       4646           False  
connectx                                                           2030-01-01 00:00:00  Getting Started   Knowledge        439           False  
competitive-data-science-predict-future-sales                      2020-12-31 23:59:00  Playground            Kudos       5886    

In [9]:
# Ion Switchingのデータをダウンロード、アンジップ
!kaggle competitions download -c liverpool-ion-switching
!unzip sample_submission.csv.zip
!unzip train.csv.zip
!unzip test.csv.zip

 91% 9.00M/9.91M [00:00<00:00, 17.0MB/s]
100% 9.91M/9.91M [00:00<00:00, 18.4MB/s]
 64% 17.0M/26.6M [00:01<00:00, 12.5MB/s]
100% 26.6M/26.6M [00:01<00:00, 16.2MB/s]
100% 4.27M/4.27M [00:00<00:00, 21.3MB/s]

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [0]:
import pandas as pd
import numpy as np

In [11]:
train =  pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test

,time,signal
0,500.0001,-2.6498
1,500.0002,-2.8494
2,500.0003,-2.8600
3,500.0004,-2.4350
4,500.0005,-2.6155
...,...,...
1999995,699.9996,-2.9092
1999996,699.9997,-2.7422
1999997,699.9998,-2.8285
1999998,699.9999,-2.9092


In [0]:
train_x = train.drop(['open_channels'], axis=1)
train_y = train['open_channels']

In [107]:
bracket = train_x[1:2]
loc = train_x.loc[1]
iloc = train_x.iloc[1]
print(type(bracket))
print(type(loc))
print(type(iloc))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [108]:
train_x.shape

(5000000, 2)

In [0]:
# 練習用
practice_x = train_x[:1000]
practice_y = train_y[:1000]

In [116]:
import pandas as pd
import numpy as np


from sklearn.model_selection import KFold

from sklearn.metrics import log_loss, f1_score, accuracy_score
# from sklearn.metrics import f1_score
import lightgbm as lgb


data = train_x[90000:100000]
y = train['open_channels'][90000:100000]
test_x = test[90000:100000]

oof = np.zeros(len(data))
pred = np.zeros(len(data))
scores = []


# Modeling
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=123)
for n_fold, (tr_idx, va_idx) in enumerate(folds.split(data, y)):
    tr_x,tr_y = data.iloc[tr_idx], y.iloc[tr_idx]
    va_x,va_y = data.iloc[va_idx], y.iloc[va_idx]

    # 特徴量と目的変数をlightgbmのデータ構造に変換する
    lgb_train = lgb.Dataset(tr_x, tr_y)
    lgb_eval = lgb.Dataset(va_x, va_y)

    # ハイパーパラメータの設定
    params =  {'objective': 'binary',
                        'seed': 71,
                        'verbose': 0,
                        'metrics': 'binary_logloss'}
    num_round = 100

    # 学習の実行
    # カテゴリ変数をパラメータで指定している
    # バリエーションデータのモデルに渡し、学習の進行と共にスコアがどう変わるかモニタリングする
    # categorical_features = ['product', 'medical_info_b2', 'medical_info_3']
    model = lgb.train(params, 
                                lgb_train, num_boost_round=num_round, 
                                # categorical_features=categorical_features,
                                valid_names=['train', 'valid'])
                                # ,valid_sets=[lgb_train, lgb_eval]) # 時間経過と共に値を確認できる
    
    # バリデーションデータでのスコア確認
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    # print(f1_score(va_y, va_pred, average='macro')
    # f1_score(va_y, va_pred,  average='macro')
    # accuracy_score(va_y, va_pred, normalize=False)
    print(f'logloss: {score: 4f}')
    # 予測
    pred += model.predict(test_x)

pred /= n_fold

logloss:  0.000012
logloss:  0.000003
logloss:  0.000260
logloss:  0.012547
logloss:  0.000050


In [0]:
!cp drive/My\ Drive/lightgbm_first.csv ./

In [0]:
lightgbm_first = pd.read_csv("lightgbm_first.csv")

In [0]:
# lightgbm_first["open_channels"].valueCounts()
lightgbm = lightgbm_first["open_channels"]

In [131]:
lightgbm.value_counts()

0     1195024
1      245879
3      138300
2      115349
4       80176
5       55640
7       50602
8       48453
6       35596
9       27416
10       7565
Name: open_channels, dtype: int64

In [0]:
data = train_x[4500000:5000000]
y1 = train['open_channels']
y = train['open_channels'][4500000:5000000]
test_x = test[4500000:5000000]

In [16]:
import pandas as pd
import numpy as np


from sklearn.model_selection import KFold

from sklearn.metrics import log_loss, f1_score, accuracy_score
# from sklearn.metrics import f1_score
import lightgbm as lgb


data = train_x[4990000:5000000]
y = train['open_channels'][4990000:5000000]
test_x = test[4990000:5000000]

oof = np.zeros(len(data))
pred = np.zeros(len(data))
scores = []


# Modeling
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=123)
for n_fold, (tr_idx, va_idx) in enumerate(folds.split(data, y)):
    tr_x,tr_y = data.iloc[tr_idx], y.iloc[tr_idx]
    va_x,va_y = data.iloc[va_idx], y.iloc[va_idx]

    # 特徴量と目的変数をlightgbmのデータ構造に変換する
    lgb_train = lgb.Dataset(tr_x, tr_y)
    lgb_eval = lgb.Dataset(va_x, va_y)

    # ハイパーパラメータの設定
    params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'metric': {'multi_logloss'},
          'num_class': 11,
          'learning_rate': 0.02,
          'num_leaves': 23,
          'min_data_in_leaf': 1,
          'num_iteration': 1000,
          'verbose': 0}
    num_round = 100

    # 学習の実行
    # カテゴリ変数をパラメータで指定している
    # バリエーションデータのモデルに渡し、学習の進行と共にスコアがどう変わるかモニタリングする
    # categorical_features = ['product', 'medical_info_b2', 'medical_info_3']
    model = lgb.train(params, 
                                lgb_train, num_boost_round=num_round, 
                                # categorical_features=categorical_features,
                                valid_names=['train', 'valid'])
                                # ,valid_sets=[lgb_train, lgb_eval]) # 時間経過と共に値を確認できる
    
    # バリデーションデータでのスコア確認
    va_pred = model.predict(va_x)
    print(va_pred)
    # score = log_loss(va_y, va_pred)
    # print(f1_score(va_y, va_pred, average='macro')
    # f1_score(va_y, va_pred,  average='macro')
    # accuracy_score(va_y, va_pred, normalize=False)
    # print(f'logloss: {score: 4f}')
    # print(va_pred)
    # 予測
    # pred += model.predict(test_x)

# pred /= n_fold

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[[2.04995911e-16 2.04995911e-16 3.39812667e-08 ... 9.75871161e-01
  8.52575577e-05 5.16768319e-07]
 [2.25643902e-16 2.25643902e-16 3.74035844e-08 ... 2.01278180e-03
  4.91157912e-06 5.68819249e-07]
 [2.09682798e-16 2.09682798e-16 3.47581904e-08 ... 9.98182813e-01
  1.21189221e-03 5.28583357e-07]
 ...
 [2.53292966e-16 2.53292966e-16 5.91532010e-08 ... 6.37253909e-03
  1.01773256e-05 9.54096475e-07]
 [1.64538863e-16 1.64538863e-16 3.84232926e-08 ... 8.98812983e-01
  6.61118078e-06 6.19780137e-07]
 [5.26302826e-16 5.26302826e-16 1.22911275e-07 ... 8.29958827e-02
  2.11468772e-05 1.98246196e-06]]
[[2.56312885e-16 2.56312885e-16 8.31649530e-08 ... 9.96152513e-01
  3.68241132e-05 7.62450228e-07]
 [3.79800957e-16 3.79800957e-16 1.23229722e-07 ... 2.38627262e-02
  9.76092595e-01 3.80707967e-06]
 [9.16789903e-16 9.16789903e-16 2.97443598e-07 ... 3.09089729e-01
  6.90802346e-01 9.26534463e-06]
 ...
 [3.76169421e-16 3.76169421e-16 1.63408635e-07 ... 9.72132515e-01
  2.78244856e-02 1.84508755e-06]

In [142]:
np.unique(pred)

array([0.02875025, 0.02883041, 0.02887368, 0.0288983 , 0.02890445,
       0.02890821, 0.02891621, 0.0289579 , 0.02898541, 0.02911544,
       0.02923517, 0.02955824, 0.02969918, 0.02980975, 0.02999833,
       0.03003651, 0.03004007, 0.03180009, 0.03211662, 0.03354366,
       0.03379505, 0.03381388, 0.03392412, 0.03401553, 0.03404165,
       0.04010917, 0.05329548, 0.06020864, 0.06053532, 0.06063539,
       0.06269652, 0.06355273, 0.08297413, 0.10693588, 0.12623509,
       0.16194932, 0.21544734, 0.26628079, 0.32086143, 0.37574649,
       0.42835838, 0.44575153, 0.46698434, 0.50390953, 0.51895706])

In [145]:
np.unique(np.round(pred).astype(np.int))

array([0, 1])

In [157]:
y1.value_counts()

0     1240152
1      985865
3      668609
2      553924
4      403410
5      277877
7      265015
8      245183
6      188112
9      136120
10      35733
Name: open_channels, dtype: int64